In [24]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [25]:
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
import pandas as pd
import os
from sklearn.metrics import balanced_accuracy_score, f1_score
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
import optuna
from pmdarima.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import joblib
import sklearn
import lightgbm as lgb
import xgboost as xgb

path_data = here("./data")
os.chdir(path_data)
data_iu_analysis = pd.read_parquet("data_iu_analysis.parquet")

In [39]:
data_iu_analysis

,w,l,w_c,l_c,sos,ps_g,pa_g,ap_pre,ap_final,tournament_make
0,23,12,12,8,8.90,74.7,68.7,13.0,21.0,1
1,21,14,9,11,8.08,70.8,66.2,NaN,NaN,1
2,12,15,7,12,12.45,69.7,69.4,NaN,NaN,0
3,20,12,9,11,10.35,71.4,66.7,NaN,NaN,0
4,19,16,8,12,10.10,71.5,67.8,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
69,20,4,12,2,9.13,75.5,67.2,NaN,4.0,1
70,23,3,17,1,8.29,81.2,69.5,NaN,1.0,1
71,16,6,9,5,5.29,73.7,66.8,NaN,NaN,0
72,19,3,12,2,5.85,64.5,53.3,NaN,7.0,0


In [26]:
train_size = round(data_iu_analysis.shape[0] * 0.8)
train_size

59

In [27]:
train, test = train_test_split(data_iu_analysis, test_size=train_size)

In [28]:
x_train = train.drop(columns="tournament_make")
x_test = test.drop(columns="tournament_make")
y_train = train["tournament_make"]
y_test = test["tournament_make"]

In [65]:
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import optuna


# Define objective function for Optuna optimization
def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "eta": trial.suggest_loguniform("eta", 1e-8, 1.0),
        "gamma": trial.suggest_loguniform("gamma", 1e-8, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 300),
        "subsample": trial.suggest_discrete_uniform("subsample", 0.1, 1, 0.05),
        "colsample_bytree": trial.suggest_discrete_uniform(
            "colsample_bytree", 0.1, 1, 0.05
        ),
    }

    # Set up TimeSeriesSplit cross-validation
    tscv = TimeSeriesSplit(n_splits=5)

    # Train XGBoost model with the given hyperparameters using cross-validation
    model = xgb.XGBRegressor(**params)
    scores = cross_val_score(
        model, x_train, y_train, cv=tscv, scoring="neg_root_mean_squared_error"
    )
    rmse = -1.0 * scores.mean()

    return rmse


# Run hyperparameter optimization using Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

[I 2023-04-14 16:28:39,249] A new study created in memory with name: no-name-0675a23d-786d-4dc1-bc4f-28181a3f6c49
/tmp/ipykernel_10128/2907057420.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
/tmp/ipykernel_10128/2907057420.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
/tmp/ipykernel_10128/2907057420.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

In [64]:
# Train final XGBoost model using the best hyperparameters found by Optuna
best_params = study.best_params
best_model = xgb.XGBRegressor(**best_params)
best_model.fit(x_train, y_train)

# Return the best model
best_model

ValueError: No trials are completed yet.

In [52]:
path_outputs = here("./outputs")
os.chdir(path_outputs)
joblib.dump(best_model, "model_iu_bball.jlib")

['model_iu_bball_lgb.jlib']

In [53]:
model_iu_bball_lgb = joblib.load("model_iu_bball_lgb.jlib")

In [54]:
prob = model_iu_bball_lgb.predict(x_test)

In [55]:
from sklearn.metrics import f1_score


def find_best_threshold(y_true, y_pred_prob):
    """
    Find the best threshold to use for converting predicted probabilities to binary predictions,
    using the F1 score as the metric to optimize for.

    :param y_true: true labels, array-like of shape (n_samples,)
    :param y_pred_prob: predicted probabilities, array-like of shape (n_samples,)
    :return: best threshold value
    """
    thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    best_f1 = 0
    best_threshold = 0
    for threshold in thresholds:
        y_pred = (y_pred_prob >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    return best_threshold

In [56]:
threshold = find_best_threshold(y_true=y_test, y_pred_prob=prob)
threshold

0.1

In [57]:
prob_threshold = np.where(prob > threshold, 1, 0)

In [58]:
balanced_accuracy = balanced_accuracy_score(y_true=y_test, y_pred=prob_threshold)
balanced_accuracy

0.5

In [59]:
f1_accuracy = f1_score(y_true=y_test, y_pred=prob_threshold)
f1_accuracy

0.7311827956989247